In [1]:
import dropbox
import os
import mp
from tqdm import tqdm

access_token = ''
dbx = dropbox.Dropbox(access_token)

In [2]:
from bs4 import BeautifulSoup

with open('Same Room Scripts - Dropbox.html') as fopen:
    html = fopen.read()
    
bs = BeautifulSoup(html)
hrefs = bs.select('a[href*=".TextGrid"]')
hrefs_text = [href['href'] for href in hrefs]

In [3]:
len(hrefs_text)

1072

In [4]:
local_directory = 'TEXT'
os.system(f'rm -rf {local_directory}')
os.system(f'mkdir {local_directory}')

0

In [5]:
def loop(hrefs):
    hrefs = hrefs[0]
    for i in tqdm(range(len(hrefs))):
        href = hrefs[i]
        while True:
            try:
                name = dbx.sharing_get_shared_link_metadata(href).name
                break
            except Exception as e:
                print(e)
        d = f'{local_directory}/{name}'
        if os.path.exists(d):
            print(f'{d} exists, skip')
            continue
        while True:
            try:
                dbx.sharing_get_shared_link_file_to_file(d, href)
                break
            except Exception as e:
                print(e)

In [7]:
loop((hrefs_text[:1],))

100%|██████████| 1/1 [00:00<00:00,  2.31it/s]

TEXT/sur_0009_1018_phns_cs-chn.TextGrid exists, skip


In [8]:
# mp.multiprocessing(hrefs_text, loop, cores = 6, returned = False)

In [9]:
with open('Same Room Audio - Dropbox.html') as fopen:
    html = fopen.read()
    
bs = BeautifulSoup(html)
hrefs = bs.select('a[href*=".wav"]')
hrefs = [href['href'] for href in hrefs]

In [10]:
from collections import defaultdict
from glob import glob
import os

speakers = {}
files = glob('TEXT/*.TextGrid')
for path in files:
    new_path = os.path.split(path)[1].replace(' ', '-').replace('.TextGrid', '')
    f = os.path.split(path)[1].replace('.TextGrid', '')
    speakers[f] = path
    
len(speakers)

1071

In [12]:
from tqdm import tqdm

directory = 'part4-same-room'

os.system(f'mkdir {directory}')
os.system(f'mkdir {directory}/wav')
os.system(f'mkdir {directory}/text')

0

In [13]:
import soundfile as sf
import textgrid

In [14]:
def split(path):
    new_path = os.path.split(path)[1].replace(' ', '-').replace('.wav', '')
    f = os.path.split(path)[1].replace('.wav', '')
    y, sr = sf.read(path)
    index = 0
    path_textgrid = speakers[f]
    tg = textgrid.TextGrid.fromFile(path_textgrid)
    for i in range(len(tg)):
        for k in range(len(tg[i])):
            try:
                splitted = y[int(sr * tg[i][k].minTime): int(sr * tg[i][k].maxTime)]
                sf.write(f'{directory}/wav/{new_path}-{i}-{k}.wav', splitted, sr)
                with open(f'{directory}/text/{new_path}-{i}-{k}.wav.txt', 'w') as fopen:
                    fopen.write(tg[i][k].mark)
            except Exception as e:
                print('nested exception', e)

In [15]:
local_directory = 'WAVE'
os.system(f'mkdir {local_directory}')

def loop(hrefs):
    hrefs = hrefs[0]
    for i in tqdm(range(len(hrefs))):
        href = hrefs[i]
        while True:
            try:
                name = dbx.sharing_get_shared_link_metadata(href).name
                break
            except:
                pass
        d = f'{local_directory}/{name}'
        if not os.path.exists(d):
            while True:
                try:
                    dbx.sharing_get_shared_link_file_to_file(d, href)
                    break
                except Exception as e:
                    print(e)
        
        try:
            split(d)
        except:
            pass
        os.remove(d)

In [16]:
loop((hrefs[:1],))

  0%|          | 0/1 [00:00<?, ?it/s]

WAVE/sur_0009_1018_phns_cs-chn.wav sur_0009_1018_phns_cs-chn.wav


100%|██████████| 1/1 [00:24<00:00, 24.12s/it]


In [18]:
mp.multiprocessing(hrefs, loop, cores = 6, returned = False)

In [19]:
wavs = glob(f'{directory}/wav/*.wav')
wavs[0]

'part4-same-room/wav/sur_0392_1785_phns_cs-chn-0-383.wav'

In [27]:
import IPython.display as ipd
ipd.Audio(wavs[5])

In [28]:
!cat {wavs[5].replace('/wav/', '/text/') + '.txt'}

<mandarin>奇怪一开始:qi guai yi kai shi</mandarin> a bit [lah] a bit <mandarin>奇怪:qi guai</mandarin> but after that <mandarin>就还好:jiu hai hao</mandarin>

In [29]:
# !tar --use-compress-program="pigz -0 -k " -cvf part4-same-room.tar.gz part4-same-room